In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from featuretools.primitives import AggregationPrimitive, make_agg_primitive
from datetime import datetime, timedelta
import re
import featuretools as ft

In [2]:
basic_B_meta = pd.read_csv('/Users/JoonH/basic_data_B_meta.csv')
basic_C_meta = pd.read_csv('/Users/JoonH/basic_data_C_meta.csv')
basic_test_meta = pd.read_csv('/Users/JoonH/basic_data_test_meta.csv')

eng_B_meta = pd.read_csv('/Users/JoonH/eng_data_B_meta.csv')
eng_C_meta = pd.read_csv('/Users/JoonH/eng_data_C_meta.csv')
eng_test_meta = pd.read_csv('/Users/JoonH/eng_data_test_meta.csv')

xgb_B_meta = pd.read_csv('/Users/JoonH/eng_data_B_meta_xgb.csv')
xgb_C_meta = pd.read_csv('/Users/JoonH/eng_data_C_meta_xgb.csv')
xgb_test_meta = pd.read_csv('/Users/JoonH/eng_data_test_meta_xgb.csv')

In [3]:
gbm_B_meta = pd.read_csv('/Users/JoonH/eng_data_gbm_B.csv')
gbm_C_meta = pd.read_csv('/Users/JoonH/eng_data_gbm_C.csv')
gbm_test_meta = pd.read_csv('/Users/JoonH/eng_data_gbm_test.csv')

In [4]:
print(gbm_B_meta.shape, gbm_C_meta.shape, gbm_test_meta.shape)

(129154, 2) (55353, 2) (48744, 2)


In [5]:
feature_spec = pd.read_csv('total_feature_matrix_spec.csv')

In [6]:
train = feature_spec[feature_spec['set'] == 'train']
test = feature_spec[feature_spec['set'] == 'test']

In [7]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)
train_labels= train['TARGET']
train, test = train.align(test, join = 'inner', axis = 1)
train['TARGET'] = train_labels

In [8]:
# Devide train data into 3: train A, B, and C
from sklearn.model_selection import train_test_split
train_A, train_B = train_test_split(train, test_size = 0.6, random_state = 3)
train_B, train_C = train_test_split(train_B, test_size = 0.3, random_state = 3)


In [9]:
print(train_A.shape, train_B.shape, train_C.shape, test.shape)

(123004, 1157) (129154, 1157) (55353, 1157) (48744, 1157)


In [10]:
# train labels for b_meta and c_meta
b_meta_labels = train_B['TARGET']
c_meta_labels = train_C['TARGET']
del train_A, train_B, train_C, feature_spec, train, test

In [13]:
basic_B_meta.head()

,Unnamed: 0,ada_basic,bag_basic,gbc_basic,rf_basic,rf_2_basic,log_basic,rid_basic,sgd_basic,gnb_basic,knn_basic
0,0,0,1,0,0,0,0,0,1,0,0
1,1,0,1,0,0,0,0,0,1,0,0
2,2,0,0,0,0,0,0,0,1,0,0
3,3,0,1,0,0,0,0,0,1,0,0
4,4,0,1,0,0,0,0,0,1,0,0


In [14]:
eng_B_meta.head()

,Unnamed: 0,ada_eng,gbc_eng,rf_1_eng,rf_2_eng,log_eng,knn_eng,svc_eng
0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,2,0.0,1.0,1.0,1.0,0.0,0.0,0.0
3,3,0.0,1.0,1.0,1.0,0.0,0.0,0.0
4,4,0.0,1.0,1.0,1.0,0.0,1.0,0.0


In [15]:
xgb_B_meta.head()

,Unnamed: 0,eng_xgb_1,eng_xgb_2,eng_xgb_3
0,0,0.219005,0.448055,0.461210
1,1,0.221092,0.456430,0.466419
2,2,0.196986,0.445541,0.460764
3,3,0.264329,0.460987,0.470842
4,4,0.245918,0.447953,0.461444


In [16]:
gbm_B_meta.head()

,Unnamed: 0,eng_gbm
0,0,0.145794
1,1,0.464633
2,2,0.130807
3,3,0.597577
4,4,0.513868


In [11]:
gbm_B_meta = gbm_B_meta.drop('Unnamed: 0',axis = 1)
gbm_C_meta = gbm_C_meta.drop('Unnamed: 0',axis = 1)
gbm_test_meta = gbm_test_meta.drop('Unnamed: 0',axis = 1)

Here we obeserve that our dataframes not unified in terms of class/proba predictions and dtype. Since only xgb predicted probability, we will convert all data to hold int binary classification.

In [12]:
def convert_proba(data, name):
    with np.nditer(data, op_flags = ['readwrite']) as it:
        for i in it:
            if i[...] < 0.5:
                i[...] = 0
            else:
                i[...] = 1
        data = pd.DataFrame(data, columns = [name])
    return data
xgb_B_meta_1 = np.array(xgb_B_meta['eng_xgb_1'])
xgb_B_meta_2 = np.array(xgb_B_meta['eng_xgb_2'])
xgb_B_meta_3 = np.array(xgb_B_meta['eng_xgb_3'])
gbm_B_meta = np.array(gbm_B_meta)

xgb_B_meta_1 = convert_proba(xgb_B_meta_1, 'eng_xgb_1')
xgb_B_meta_2 = convert_proba(xgb_B_meta_2, 'eng_xgb_2')
xgb_B_meta_3 = convert_proba(xgb_B_meta_3, 'eng_xgb_3')
gbm_B_meta = convert_proba(gbm_B_meta, 'eng_gbm')

xgb_B_meta = pd.concat([xgb_B_meta_1, xgb_B_meta_2, xgb_B_meta_3], axis = 1)
gbm_B_meta = pd.DataFrame(gbm_B_meta, columns = ['eng_gbm'])
xgb_B_meta.head()

,eng_xgb_1,eng_xgb_2,eng_xgb_3
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


In [13]:
# worked correctly, let's try this for C and test meta data
xgb_C_meta_1 = np.array(xgb_C_meta['eng_xgb_1'])
xgb_C_meta_2 = np.array(xgb_C_meta['eng_xgb_2'])
xgb_C_meta_3 = np.array(xgb_C_meta['eng_xgb_3'])
gbm_C_meta = np.array(gbm_C_meta)

xgb_C_meta_1 = convert_proba(xgb_C_meta_1, 'eng_xgb_1')
xgb_C_meta_2 = convert_proba(xgb_C_meta_2, 'eng_xgb_2')
xgb_C_meta_3 = convert_proba(xgb_C_meta_3, 'eng_xgb_3')
gbm_C_meta = convert_proba(gbm_C_meta, 'eng_gbm')

gbm_C_meta = pd.DataFrame(gbm_C_meta, columns = ['eng_gbm'])
xgb_C_meta = pd.concat([xgb_C_meta_1, xgb_C_meta_2, xgb_C_meta_3], axis = 1)

xgb_test_meta_1 = np.array(xgb_test_meta['eng_xgb_1'])
xgb_test_meta_2 = np.array(xgb_test_meta['eng_xgb_2'])
xgb_test_meta_3 = np.array(xgb_test_meta['eng_xgb_3'])
gbm_test_meta = np.array(gbm_test_meta)

xgb_test_meta_1 = convert_proba(xgb_test_meta_1, 'eng_xgb_1')
xgb_test_meta_2 = convert_proba(xgb_test_meta_2, 'eng_xgb_2')
xgb_test_meta_3 = convert_proba(xgb_test_meta_3, 'eng_xgb_3')
gbm_test_meta = convert_proba(gbm_test_meta, 'eng_gbm')

xgb_test_meta = pd.concat([xgb_test_meta_1, xgb_test_meta_2, xgb_test_meta_3], axis = 1)
gbm_test_meta = pd.DataFrame(gbm_test_meta, columns = ['eng_gbm'])

Let's put the train data alltogether, by B_meta, C_meta and test_meta

In [14]:
B_meta = pd.concat([basic_B_meta.drop('Unnamed: 0', axis = 1), 
                    eng_B_meta.drop('Unnamed: 0', axis = 1),
                    xgb_B_meta, gbm_B_meta], axis = 1)

C_meta = pd.concat([basic_C_meta.drop('Unnamed: 0', axis = 1), 
                    eng_C_meta.drop('Unnamed: 0', axis = 1),
                    xgb_C_meta, gbm_C_meta], axis = 1) 

test_meta = pd.concat([basic_test_meta.drop('Unnamed: 0', axis = 1), 
                    eng_test_meta.drop('Unnamed: 0', axis = 1),
                    xgb_test_meta, gbm_test_meta], axis = 1) 

In [22]:
B_meta.head()

,ada_basic,bag_basic,gbc_basic,rf_basic,rf_2_basic,log_basic,rid_basic,sgd_basic,gnb_basic,knn_basic,...,gbc_eng,rf_1_eng,rf_2_eng,log_eng,knn_eng,svc_eng,eng_xgb_1,eng_xgb_2,eng_xgb_3,eng_gbm
0,0,1,0,0,0,0,0,1,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0,0,0,0,0,1,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,1,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,0,0,0,0,1,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,1,0,0,0,0,0,1,0,0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [23]:
C_meta.head()

,ada_basic,bag_basic,gbc_basic,rf_basic,rf_2_basic,log_basic,rid_basic,sgd_basic,gnb_basic,knn_basic,...,gbc_eng,rf_1_eng,rf_2_eng,log_eng,knn_eng,svc_eng,eng_xgb_1,eng_xgb_2,eng_xgb_3,eng_gbm
0,0,1,0,0,0,0,0,1,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0,0,0,0,0,1,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0,0,0,0,0,1,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,0,0,0,0,1,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,0,1,0,0,0,0,0,1,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
test_meta.head()

,ada_basic,bag_basic,gbc_basic,rf_basic,rf_2_basic,log_basic,rid_basic,sgd_basic,gnb_basic,knn_basic,...,gbc_eng,rf_1_eng,rf_2_eng,log_eng,knn_eng,svc_eng,eng_xgb_1,eng_xgb_2,eng_xgb_3,eng_gbm
0,0,0,0,0,0,0,0,1,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0,0,0,0,0,1,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,1,0,0,0,0,0,1,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,0,0,0,0,1,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,0,0,0,0,0,1,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
B_meta = B_meta.astype(int)
C_meta = C_meta.astype(int)
test_meta = test_meta.astype(int)

#### LightGBM model 

In [16]:
from sklearn.preprocessing import MinMaxScaler, Imputer
from sklearn.model_selection import KFold
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc

In [50]:
# thank you Will Koehrsen for an amazing kernel / this method!

def model(features, labels , test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    #train_ids = features['SK_ID_CURR']
    #test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    #labels = features['TARGET']
    
    # Remove the ids and target
    #features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    #features = features.drop(columns = ['TARGET'])
    #test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=1000, objective = 'binary', 
                                   is_unbalanced = True, learning_rate = 0.0001, 
                                   reg_alpha = 0.0, reg_lambda = 0.0, 
                                   subsample = 0.5, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    #submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    submission = pd.DataFrame({'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

Hyperparameter tuning

In [59]:
model = lgb.LGBMClassifier()
print (model.get_params())

{'min_child_weight': 0.001, 'objective': None, 'reg_alpha': 0.0, 'subsample_for_bin': 200000, 'colsample_bytree': 1.0, 'random_state': None, 'max_depth': -1, 'subsample_freq': 0, 'silent': True, 'min_child_samples': 20, 'num_leaves': 31, 'n_jobs': -1, 'min_split_gain': 0.0, 'reg_lambda': 0.0, 'boosting_type': 'gbdt', 'learning_rate': 0.1, 'class_weight': None, 'subsample': 1.0, 'n_estimators': 100}


In [60]:
train_set = lgb.Dataset(data = B_meta, label = b_meta_labels.astype(int).reset_index(drop=True))
test_set = lgb.Dataset(data = C_meta, label = c_meta_labels.astype(int).reset_index(drop=True))

In [61]:
def objective(hyperparameters, iteration):
    """Objective function for grid and random search. Returns
       the cross validation score from a set of hyperparameters."""
    
    # Number of estimators will be found using early stopping
    if 'n_estimators' in hyperparameters.keys():
        del hyperparameters['n_estimators']
    
     # Perform n_folds cross validation
    cv_results = lgb.cv(hyperparameters, train_set, num_boost_round = 10000, nfold = 5, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = 42)
    
    # results to retun
    score = cv_results['auc-mean'][-1]
    estimators = len(cv_results['auc-mean'])
    hyperparameters['n_estimators'] = estimators 
    
    return [score, hyperparameters, iteration]

In [82]:
# Performing gridsearch with C_meta
# Hyperparameter grid
param_grid = {
    'objective' : ['binary'],
    'boosting': ['gbdt', 'gbrt','goss', 'dart'],
    'subsample_for_bin': list(range(50, 300000)),
    'min_child_samples': list(range(10, 750)),
    'reg_alpha': list(np.linspace(0, 1)),
    'reg_lambda': list(np.linspace(0, 1)),
    'colsample_bytree': list(np.linspace(0.6, 1, 10)),
    'subsample': list(np.linspace(0.5, 1, 10)),
    'is_unbalance': [True, False],
    'max_bin' : list(range(255,100000)),
    'learning_rate': list(np.logspace(np.log10(0.00000001), np.log10(0.5), base = 10, num = 1000)),
    'num_leaves': list(range(2, 200000)),
    'n_estimators' : list(range(1,2000000))
}

MAX_EVALS = 5

import itertools

def grid_search(param_grid, max_evals = MAX_EVALS):
    """Grid search algorithm (with limit on max evals)"""
    
    # Dataframe to store results
    results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                              index = list(range(MAX_EVALS)))
    
    # https://codereview.stackexchange.com/questions/171173/list-all-possible-permutations-from-a-python-dictionary-of-lists
    keys, values = zip(*param_grid.items())
    
    i = 0
    
    # Iterate through every possible combination of hyperparameters
    for v in itertools.product(*values):
        
        # Create a hyperparameter dictionary
        hyperparameters = dict(zip(keys, v))
        
        # Set the subsample ratio accounting for boosting type
        hyperparameters['subsample'] = 1.0 if hyperparameters['boosting'] == 'goss' else hyperparameters['subsample']
        
        # Evalute the hyperparameters
        eval_results = objective(hyperparameters, i)
        
        results.loc[i, :] = eval_results
        
        i += 1
        
        # Normally would not limit iterations
        if i > MAX_EVALS:
            break
       
    # Sort with best score on top
    results.sort_values('score', ascending = False, inplace = True)
    results.reset_index(inplace = True)
    
    return results   

In [83]:
grid_results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                              index = list(range(MAX_EVALS)))

grid_results = grid_search(param_grid)

print('The best validation score was {:.5f}'.format(grid_results.loc[0, 'score']))
print('\nThe best hyperparameters were:')

import pprint
pprint.pprint(grid_results.loc[0, 'params'])

The best validation score was 0.71399

The best hyperparameters were:
{'boosting': 'gbdt',
 'colsample_bytree': 0.6,
 'is_unbalance': True,
 'learning_rate': 1e-08,
 'max_bin': 255,
 'metric': 'auc',
 'min_child_samples': 10,
 'n_estimators': 25,
 'num_leaves': 2,
 'objective': 'binary',
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 0.5,
 'subsample_for_bin': 50,
 'verbose': 1}


#### Bagging with lightgbm

In [47]:
def model_bag(features, labels , test_features, random_seed, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    #train_ids = features['SK_ID_CURR']
    #test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    #labels = features['TARGET']
    
    # Remove the ids and target
    #features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    #features = features.drop(columns = ['TARGET'])
    #test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators = 100, objective = 'binary', 
                                   is_unbalance = True, learning_rate = 1e-03, 
                                   reg_alpha = 0.0, reg_lambda = 0.0, max_bin = 255,
                                   subsample = 0.5, n_jobs = -1, random_state = seed,
                                   metric = 'auc', min_child_samples = 10, num_leaves = 10,
                                   subsample_for_bin = 50, olsample_bytree = 0.6,)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    #submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    #submission = pd.DataFrame({'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return test_predictions #submission #, feature_importances, metrics

In [40]:
bags = 10
seed = 1
bagged_predictions = np.zeros(test_meta.shape[0])
bag_train_data = pd.concat([B_meta, C_meta], axis = 0)
bag_train_labels = pd.concat([b_meta_labels.astype(int).reset_index(drop=True),
                              c_meta_labels.astype(int).reset_index(drop=True)],
                             axis = 0)
for n in range (0, bags):
    preds = model_bag(bag_train_data, bag_train_labels.reset_index(drop=True), test_meta, seed + n)
    bagged_predictions+=preds
    print('predictions gathered from model.')
    
bagged_predictions/=bags

Training Data Shape:  (184507, 21)
Testing Data Shape:  (48744, 21)
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.721917	train's auc: 0.718971
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.717717	train's auc: 0.716788
Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.715062	train's auc: 0.722375
Early stopping, best iteration is:
[116]	valid's auc: 0.715192	train's auc: 0.722343
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.713506	train's auc: 0.716685
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.721366	train's auc: 0.71904
predictions gathered from model.
Training Data Shape:  (184507, 21)
Testing Data Shape:  (48744, 21)
Training until validation scores don't improve for

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.717717	train's auc: 0.716788
Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.715062	train's auc: 0.722375
Early stopping, best iteration is:
[116]	valid's auc: 0.715192	train's auc: 0.722343
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.713506	train's auc: 0.716685
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.721366	train's auc: 0.71904
predictions gathered from model.


In [41]:
bagged_predictions

array([0.08642378, 0.09532946, 0.0864232 , ..., 0.0864232 , 0.0864232 ,
       0.09532946])

In [42]:
bagged_predictions = convert_proba(bagged_predictions, 'TARGET')
bagged_predictions.head()

,TARGET
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [45]:
ids = pd.read_csv('/Users/JoonH/credit_stacking.csv').drop('TARGET', axis = 1)
submission = pd.concat([ids, bagged_predictions], axis = 1)
submission.to_csv('/Users/JoonH/credit_bagged.csv', index = False)

## Final Predictions
The model is tuned to fit the following hyperparameters, and are now ready to make the test predictions.

In [67]:
# thank you Will Koehrsen for an amazing kernel / this method!

def model(features, labels , test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    #train_ids = features['SK_ID_CURR']
    #test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    #labels = features['TARGET']
    
    # Remove the ids and target
    #features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    #features = features.drop(columns = ['TARGET'])
    #test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=1000, objective = 'binary', 
                                   is_unbalanced = True, learning_rate = 0.0001, 
                                   reg_alpha = 0.0, reg_lambda = 0.0, 
                                   subsample = 0.5, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    #submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    submission = pd.DataFrame({'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [64]:
stacking_train = pd.concat([B_meta, C_meta], axis = 0)
stacking_train_labels = pd.concat([b_meta_labels.astype(int).reset_index(drop=True),
                              c_meta_labels.astype(int).reset_index(drop=True)],
                             axis = 0)

submission, fi, metrics = model(stacking_train,stacking_train_labels.reset_index(drop=True), test_meta)
print('Baseline metrics')
print(metrics)

Training Data Shape:  (184507, 21)
Testing Data Shape:  (48744, 21)
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.721407	train's auc: 0.721889
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[80]	valid's auc: 0.716875	train's auc: 0.723575
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.72195	train's auc: 0.720083
Baseline metrics
      fold     train     valid
0        0  0.721889  0.721407
1        1  0.723575  0.716875
2        2  0.720083  0.721950
3  overall  0.721849  0.604058


In [65]:
testy = pd.read_csv('/Users/JoonH/Desktop/Home_Credit_Challenge/all/application_test.csv')

In [66]:
submission = pd.concat([testy['SK_ID_CURR'], submission], axis = 1)

submission.to_csv('credit_stacking.csv', index = False)

### Conclusion

Despite forming stacking and grid search to validate our approach, it appears to be that we haven't gone beyond our major score of 0.756 but even failed to reach that point! It is also possible that the meta model is experiencing some overfitting issues, as with more grid search with more options the model's public LB score began to drop to 0.68 from 0.69.

It may be concluded that our model needs a more less-overfittable approach with use of other methods to further push its accuracy. 

### But wait... there are more ways, right? 
#### If this is all I could do, I should be #1 (But unfortunately I am not)

Indeed, my approach was very amateurish and it may and definitely have undergone some assumptions that ignored many points to consider for a better, more accurate solution. Here are some of the following: 

If my data was time-sensitive, then we must make sure that our train data holds elements at a generation before our cross validation and test dataset. 

I used simple Out Of Fold validation, which is more prone to overfitting. We should try to use K-fold cross validation with the right value of K to prevent target leakage. 

We can have more various models in the early layers. 

We should also consider more feature engineering for our meta-models such as pairwise differences between meta features, row-wise statistics,etc. 

Last but not least, we should try to run StackNet and get it working for our solution, as it is one of the state-of-art method that will allow us to score much higher on the leaderboard. 
